In [1]:
import pandas as pd 
import numpy as np

In [5]:
org_file = pd.DataFrame(pd.read_csv('/Users/rushil/ichseg/synthstrip/annotations.csv'))
org_craniotomy_images = org_file[org_file['7 - Craniotomy'] == 'yes']['Filename'].tolist()
org_cta_images = org_file[org_file['5 - CTA'] == 'yes']['Filename'].tolist()
org_artifact_images = org_file[org_file['6 - Noisy Artifacts'] == 'yes']['Filename'].tolist()

methods = ['v1', 'robust', 'synthstrip', 'hdctbet', 'ctbet', 'brainchop', 'dockerctbet']
types = ['craniotomy', 'cta', 'artifact']

for method in methods:
    print(f"Processing method: {method}")
    # Read the CSV once per method
    final_csv = pd.DataFrame(pd.read_csv(f'/Users/rushil/ichseg/{method}/new/{method}_final.csv'))
    
    # Add all type columns to the same DataFrame
    for type in types:
        print(f"  Adding {type} column")
        final_csv[type] = final_csv['Filename'].apply(lambda x: 1 if x in eval(f'org_{type}_images') else 0)
    
    # Save once per method with all columns added
    final_csv.to_csv(f'/Users/rushil/ichseg/{method}/new/{method}_final_v2.csv', index=False)

Processing method: v1
  Adding craniotomy column
  Adding cta column
  Adding artifact column
Processing method: robust
  Adding craniotomy column
  Adding cta column
  Adding artifact column
Processing method: synthstrip
  Adding craniotomy column
  Adding cta column
  Adding artifact column
Processing method: hdctbet
  Adding craniotomy column
  Adding cta column
  Adding artifact column
Processing method: ctbet
  Adding craniotomy column
  Adding cta column
  Adding artifact column
Processing method: brainchop
  Adding craniotomy column
  Adding cta column
  Adding artifact column
Processing method: dockerctbet
  Adding craniotomy column
  Adding cta column
  Adding artifact column


In [9]:
methods = ['v1','robust','hdctbet','ctbet','brainchop','dockerctbet']

exclude_prefixes = ("6046","6084","6096","6246","6315","6342","6499")
brainchop_exclude = [
    '6109-317_20150302_0647_ct.png','6142-308_20150610_0707_ct.png','6193-324_20150924_1431_ct.png',
    '6257-335_20160118_1150_ct.png','6418-193_20161228_1248_ct.png','6470-296_20170602_0607_ct.png',
    '6480-154_20170622_0937_ct.png'
]
dockerctbet_exclude = [
    '6109-317_20150302_0647_ct.png','6142-308_20150610_0707_ct.png','6193-324_20150924_1431_ct.png',
    '6257-335_20160118_1150_ct.png','6418-193_20161228_1248_ct.png','6470-296_20170602_0607_ct.png',
    '6480-154_20170622_0937_ct.png'
]

rows = []
for method in methods:
    df = pd.read_csv(f"/Users/rushil/downloads/{method}/annotations.csv")

    # normalize filename and patient id
    df["basename"] = df["Filename"].astype(str).str.split("/").str[-1]
    df["pid"] = df["basename"].str.split("_").str[0]

    # method-specific file exclusions
    if method == "brainchop":
        df = df[~df["basename"].isin(brainchop_exclude)]
    if method == "dockerctbet":
        df = df[~df["basename"].isin(dockerctbet_exclude)]

    # global withdrawn patient exclusions
    df = df[~df["pid"].str.startswith(exclude_prefixes)]

    # flags are 'yes'/'no' strings; compare case-insensitively
    def yes(col): return df[col].astype(str).str.strip().str.lower().eq("yes")

    cta = df[yes('5 - CTA')]
    art = df[yes('6 - Noisy Artifacts')]
    cran = df[yes('7 - Craniotomy')]

    rows.append({
        "method": method,
        "cta_scans": len(cta), "cta_subjects": cta["pid"].nunique(),
        "artifact_scans": len(art), "artifact_subjects": art["pid"].nunique(),
        "craniotomy_scans": len(cran), "craniotomy_subjects": cran["pid"].nunique(),
        "total_scans_after_excl": len(df), "total_subjects_after_excl": df["pid"].nunique(),
    })

summary = pd.DataFrame(rows)
print(summary)


        method  cta_scans  cta_subjects  artifact_scans  artifact_subjects  \
0           v1         10            10              53                 46   
1       robust         10            10              53                 46   
2      hdctbet         10            10              53                 46   
3        ctbet          7             7              28                 27   
4    brainchop         10            10              53                 46   
5  dockerctbet         10            10              53                 46   

   craniotomy_scans  craniotomy_subjects  total_scans_after_excl  \
0                40                   14                    5062   
1                40                   14                    5062   
2                40                   14                    5062   
3                22                    7                    2496   
4                40                   14                    5062   
5                40                   14     

**ALL RESULTS**: 

In [10]:
methods = ['CTBET', 'Robust-CTBET', 'SynthStrip', 'HD-CTBET', 'CT_BET', 'Brainchop', 'CTbet_Docker']
exclude_prefixes = ("6046", "6084", "6096", "6246", "6315", "6342", "6499")

brainchop_exclude = ['6109-317_20150302_0647_ct.png', '6142-308_20150610_0707_ct.png', '6193-324_20150924_1431_ct.png', '6257-335_20160118_1150_ct.png',
                     '6418-193_20161228_1248_ct.png', '6470-296_20170602_0607_ct.png', '6480-154_20170622_0937_ct.png']

dockerctbet_exclude = ['6109-317_20150302_0647_ct.png', '6142-308_20150610_0707_ct.png', '6193-324_20150924_1431_ct.png', '6257-335_20160118_1150_ct.png',
                       '6418-193_20161228_1248_ct.png', '6470-296_20170602_0607_ct.png', '6480-154_20170622_0937_ct.png']

all_results = []

for method in methods:
    print(f"Processing method: {method}")
    # Read the single CSV file with the new columns
    csv_file = pd.read_csv(f'/Users/rushil/brain_extraction/data/qc/{method}/{method}_final_v2.csv')
    if method == 'Brainchop':
        csv_file = csv_file[~csv_file['Filename'].isin(brainchop_exclude)]
        print(f"Filtered out {len(brainchop_exclude)} files from brainchop")
        
    if method == 'CTbet_Docker':
        csv_file = csv_file[~csv_file['Filename'].isin(dockerctbet_exclude)]
        print(f"Filtered out {len(dockerctbet_exclude)} files from dockerctbet")    

    
    # Apply exclusions based on filename prefixes
    mask = ~csv_file['Filename'].str.startswith(exclude_prefixes)
    print(f"Excluding {len(csv_file) - mask.sum()} files from {method}")
    csv_file = csv_file[mask]
    
    # Define the failure columns to check
    failure_columns = ['1 - Volumetrics', '2 - Registration', '3 - DL']
    available_columns = [col for col in failure_columns if col in csv_file.columns]
    print(f"Available failure columns for {method}: {available_columns}")
    
    if method != 'CT_BET':
        total_count = 5062
    else:
        total_count = 2403
    
    # Count failures for each individual column
    volumetrics_count = len(csv_file[csv_file['1 - Volumetrics'] == 'yes']) if '1 - Volumetrics' in available_columns else 0
    registration_count = len(csv_file[csv_file['2 - Registration'] == 'yes']) if '2 - Registration' in available_columns else 0
    dl_count = len(csv_file[csv_file['3 - DL'] == 'yes']) if '3 - DL' in available_columns else 0
    
    # Count total failures (any failure in any column)
    total_failures_count = len(csv_file[csv_file[available_columns].eq('yes').any(axis=1)]) if available_columns else 0
    
    # Count multiple failures (failures in 2 or more columns)
    yes_counts_per_row = csv_file[available_columns].eq('yes').sum(axis=1) if available_columns else pd.Series([0] * len(csv_file))
    multiple_failures_count = len(csv_file[yes_counts_per_row >= 2])
    
    # Create results dataframe
    df = pd.DataFrame({
        'Method': [method],
        'Total_Count': [total_count],
        'Total_Failure_Count': [total_failures_count],
        'Total_Failure_Rate': [total_failures_count / total_count * 100 if total_count > 0 else 0],
        'Volumetrics_Count': [volumetrics_count],
        'Volumetrics_Rate': [volumetrics_count / total_count * 100 if total_count > 0 else 0],
        'Registration_Count': [registration_count],
        'Registration_Rate': [registration_count / total_count * 100 if total_count > 0 else 0],
        'DL_Count': [dl_count],
        'DL_Rate': [dl_count / total_count * 100 if total_count > 0 else 0],
        'Multiple_Failures_Count': [multiple_failures_count],
        'Multiple_Failures_Rate': [multiple_failures_count / total_count * 100 if total_count > 0 else 0]
    })
    
    all_results.append(df)

# Combine all results
final_results = pd.concat(all_results, ignore_index=True)

# Define column order
column_order = [
    "Method",
    "Total_Count",
    "Total_Failure_Count", "Total_Failure_Rate",
    "Volumetrics_Count", "Volumetrics_Rate",
    "Registration_Count", "Registration_Rate",
    "DL_Count", "DL_Rate",
    "Multiple_Failures_Count", "Multiple_Failures_Rate"
]

final_results = final_results[column_order]
final_results.to_csv('/Users/rushil/brain_extraction/results/qualitative/scan_lvl/Rushil_QC_results.csv', index=False)
print("Results saved to CSV file")
final_results

Processing method: CTBET
Excluding 2 files from CTBET
Available failure columns for CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing method: Robust-CTBET
Excluding 0 files from Robust-CTBET
Available failure columns for Robust-CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing method: SynthStrip
Excluding 40 files from SynthStrip
Available failure columns for SynthStrip: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing method: HD-CTBET
Excluding 15 files from HD-CTBET
Available failure columns for HD-CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing method: CT_BET
Excluding 5 files from CT_BET
Available failure columns for CT_BET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing method: Brainchop
Filtered out 7 files from brainchop
Excluding 5 files from Brainchop
Available failure columns for Brainchop: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing method: CTbet_Docker
Filtered out 7 files fro

,Method,Total_Count,Total_Failure_Count,Total_Failure_Rate,Volumetrics_Count,Volumetrics_Rate,Registration_Count,Registration_Rate,DL_Count,DL_Rate,Multiple_Failures_Count,Multiple_Failures_Rate
0,CTBET,5062,311,6.143817,308,6.084552,188,3.713947,207,4.089293,207,4.089293
1,Robust-CTBET,5062,15,0.296326,15,0.296326,9,0.177795,10,0.197550,10,0.197550
2,SynthStrip,5062,815,16.100356,808,15.962070,262,5.175820,349,6.894508,383,7.566179
3,HD-CTBET,5062,263,5.195575,263,5.195575,0,0.000000,0,0.000000,0,0.000000
4,CT_BET,2403,790,32.875572,787,32.750728,536,22.305452,194,8.073242,591,24.594257
5,Brainchop,5062,456,9.008297,456,9.008297,296,5.847491,75,1.481628,296,5.847491
6,CTbet_Docker,5062,78,1.540893,78,1.540893,30,0.592651,30,0.592651,49,0.967997


**CRAINOTOMY RESULTS**:

In [11]:
methods = ['CTBET', 'Robust-CTBET', 'SynthStrip', 'HD-CTBET', 'CT_BET', 'Brainchop', 'CTbet_Docker']
exclude_prefixes = ("6046", "6084", "6096", "6246", "6315", "6342", "6499")

brainchop_exclude = ['6109-317_20150302_0647_ct.png', '6142-308_20150610_0707_ct.png', '6193-324_20150924_1431_ct.png', '6257-335_20160118_1150_ct.png',
                     '6418-193_20161228_1248_ct.png', '6470-296_20170602_0607_ct.png', '6480-154_20170622_0937_ct.png']

dockerctbet_exclude = ['6109-317_20150302_0647_ct.png', '6142-308_20150610_0707_ct.png', '6193-324_20150924_1431_ct.png', '6257-335_20160118_1150_ct.png',
                       '6418-193_20161228_1248_ct.png', '6470-296_20170602_0607_ct.png', '6480-154_20170622_0937_ct.png']

craniotomy_results = []

for method in methods:
    print(f"Processing craniotomy results for method: {method}")
    # Read the CSV file with the new columns
    csv_file = pd.read_csv(f'/Users/rushil/brain_extraction/data/qc/{method}/{method}_final_v2.csv')
    
    if method == 'brainchop':
        csv_file = csv_file[~csv_file['Filename'].isin(brainchop_exclude)]
        print(f"Filtered out {len(brainchop_exclude)} files from brainchop")
        
    if method == 'CTbet_Docker':
        csv_file = csv_file[~csv_file['Filename'].isin(dockerctbet_exclude)]
        print(f"Filtered out {len(dockerctbet_exclude)} files from dockerctbet")    

    # Apply exclusions based on filename prefixes
    mask = ~csv_file['Filename'].str.startswith(exclude_prefixes)
    csv_file = csv_file[mask]
    
    # Filter for craniotomy cases only
    craniotomy_cases = csv_file[csv_file['craniotomy'] == 1]
    print(f"Found {len(craniotomy_cases)} craniotomy cases for {method}")
    
    # Define the failure columns to check
    failure_columns = ['1 - Volumetrics', '2 - Registration', '3 - DL']
    available_columns = [col for col in failure_columns if col in craniotomy_cases.columns]
    print(f"Available failure columns for {method}: {available_columns}")
    
    # Set total count based on method
    if method != 'CT_BET':
        total_count = 40
    else:
        total_count = 22
    
    # Count failures for each individual column
    volumetrics_count = len(craniotomy_cases[craniotomy_cases['1 - Volumetrics'] == 'yes']) if '1 - Volumetrics' in available_columns else 0
    registration_count = len(craniotomy_cases[craniotomy_cases['2 - Registration'] == 'yes']) if '2 - Registration' in available_columns else 0
    dl_count = len(craniotomy_cases[craniotomy_cases['3 - DL'] == 'yes']) if '3 - DL' in available_columns else 0
    
    # Count total failures (any failure in any column)
    total_failures_count = len(craniotomy_cases[craniotomy_cases[available_columns].eq('yes').any(axis=1)]) if available_columns else 0
    
    # Count multiple failures (failures in 2 or more columns)
    yes_counts_per_row = craniotomy_cases[available_columns].eq('yes').sum(axis=1) if available_columns else pd.Series([0] * len(craniotomy_cases))
    multiple_failures_count = len(craniotomy_cases[yes_counts_per_row >= 2])
    
    # Create results dataframe
    df = pd.DataFrame({
        'Method': [method],
        'Total_Count': [total_count],
        'Total_Failure_Count': [total_failures_count],
        'Total_Failure_Rate': [total_failures_count / total_count * 100 if total_count > 0 else 0],
        'Volumetrics_Count': [volumetrics_count],
        'Volumetrics_Rate': [volumetrics_count / total_count * 100 if total_count > 0 else 0],
        'Registration_Count': [registration_count],
        'Registration_Rate': [registration_count / total_count * 100 if total_count > 0 else 0],
        'DL_Count': [dl_count],
        'DL_Rate': [dl_count / total_count * 100 if total_count > 0 else 0],
        'Multiple_Failures_Count': [multiple_failures_count],
        'Multiple_Failures_Rate': [multiple_failures_count / total_count * 100 if total_count > 0 else 0]
    })
    
    craniotomy_results.append(df)

# Combine all craniotomy results
final_craniotomy_results = pd.concat(craniotomy_results, ignore_index=True)

# Define column order
column_order = [
    "Method",
    "Total_Count",
    "Total_Failure_Count", "Total_Failure_Rate",
    "Volumetrics_Count", "Volumetrics_Rate",
    "Registration_Count", "Registration_Rate",
    "DL_Count", "DL_Rate",
    "Multiple_Failures_Count", "Multiple_Failures_Rate"
]

final_craniotomy_results = final_craniotomy_results[column_order]
final_craniotomy_results.to_csv('/Users/rushil/brain_extraction/results/qualitative/scan_lvl/Rushil_QC_craniotomy_results.csv', index=False)
print("Craniotomy results saved to CSV file")
final_craniotomy_results

Processing craniotomy results for method: CTBET
Found 7 craniotomy cases for CTBET
Available failure columns for CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing craniotomy results for method: Robust-CTBET
Found 0 craniotomy cases for Robust-CTBET
Available failure columns for Robust-CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing craniotomy results for method: SynthStrip
Found 31 craniotomy cases for SynthStrip
Available failure columns for SynthStrip: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing craniotomy results for method: HD-CTBET
Found 10 craniotomy cases for HD-CTBET
Available failure columns for HD-CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing craniotomy results for method: CT_BET
Found 18 craniotomy cases for CT_BET
Available failure columns for CT_BET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing craniotomy results for method: Brainchop
Found 9 craniotomy cases for Brainchop
Availabl

,Method,Total_Count,Total_Failure_Count,Total_Failure_Rate,Volumetrics_Count,Volumetrics_Rate,Registration_Count,Registration_Rate,DL_Count,DL_Rate,Multiple_Failures_Count,Multiple_Failures_Rate
0,CTBET,40,4,10.000000,4,10.000000,2,5.000000,2,5.000000,3,7.500000
1,Robust-CTBET,40,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
2,SynthStrip,40,18,45.000000,18,45.000000,11,27.500000,15,37.500000,16,40.000000
3,HD-CTBET,40,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
4,CT_BET,22,15,68.181818,15,68.181818,4,18.181818,13,59.090909,13,59.090909
5,Brainchop,40,4,10.000000,4,10.000000,3,7.500000,0,0.000000,3,7.500000
6,CTbet_Docker,40,5,12.500000,5,12.500000,0,0.000000,0,0.000000,0,0.000000


**CTA RESULTS**:

In [12]:
methods = ['CTBET', 'Robust-CTBET', 'SynthStrip', 'HD-CTBET', 'CT_BET', 'Brainchop', 'CTbet_Docker']
exclude_prefixes = ("6046", "6084", "6096", "6246", "6315", "6342", "6499")

brainchop_exclude = ['6109-317_20150302_0647_ct.png', '6142-308_20150610_0707_ct.png', '6193-324_20150924_1431_ct.png', '6257-335_20160118_1150_ct.png',
                     '6418-193_20161228_1248_ct.png', '6470-296_20170602_0607_ct.png', '6480-154_20170622_0937_ct.png']

dockerctbet_exclude = ['6109-317_20150302_0647_ct.png', '6142-308_20150610_0707_ct.png', '6193-324_20150924_1431_ct.png', '6257-335_20160118_1150_ct.png',
                       '6418-193_20161228_1248_ct.png', '6470-296_20170602_0607_ct.png', '6480-154_20170622_0937_ct.png']

cta_results = []

for method in methods:
    print(f"Processing CTA results for method: {method}")
    # Read the CSV file with the new columns
    csv_file = pd.read_csv(f'/Users/rushil/brain_extraction/data/qc/{method}/{method}_final_v2.csv')
    
    if method == 'Brainchop':
        csv_file = csv_file[~csv_file['Filename'].isin(brainchop_exclude)]
        print(f"Filtered out {len(brainchop_exclude)} files from brainchop")
        
    if method == 'CTbet_Docker':
        csv_file = csv_file[~csv_file['Filename'].isin(dockerctbet_exclude)]
        print(f"Filtered out {len(dockerctbet_exclude)} files from dockerctbet")    

    # Apply exclusions based on filename prefixes
    mask = ~csv_file['Filename'].str.startswith(exclude_prefixes)
    csv_file = csv_file[mask]
    
    # Filter for CTA cases only
    cta_cases = csv_file[csv_file['cta'] == 1]
    print(f"Found {len(cta_cases)} CTA cases for {method}")
    
    # Define the failure columns to check
    failure_columns = ['1 - Volumetrics', '2 - Registration', '3 - DL']
    available_columns = [col for col in failure_columns if col in cta_cases.columns]
    print(f"Available failure columns for {method}: {available_columns}")
    
    # Set total count based on method
    if method != 'CT_BET':
        total_count = 10
    else:
        total_count = 7
    
    # Count failures for each individual column
    volumetrics_count = len(cta_cases[cta_cases['1 - Volumetrics'] == 'yes']) if '1 - Volumetrics' in available_columns else 0
    registration_count = len(cta_cases[cta_cases['2 - Registration'] == 'yes']) if '2 - Registration' in available_columns else 0
    dl_count = len(cta_cases[cta_cases['3 - DL'] == 'yes']) if '3 - DL' in available_columns else 0
    
    # Count total failures (any failure in any column)
    total_failures_count = len(cta_cases[cta_cases[available_columns].eq('yes').any(axis=1)]) if available_columns else 0
    
    # Count multiple failures (failures in 2 or more columns)
    yes_counts_per_row = cta_cases[available_columns].eq('yes').sum(axis=1) if available_columns else pd.Series([0] * len(cta_cases))
    multiple_failures_count = len(cta_cases[yes_counts_per_row >= 2])
    
    # Create results dataframe
    df = pd.DataFrame({
        'Method': [method],
        'Total_Count': [total_count],
        'Total_Failure_Count': [total_failures_count],
        'Total_Failure_Rate': [total_failures_count / total_count * 100 if total_count > 0 else 0],
        'Volumetrics_Count': [volumetrics_count],
        'Volumetrics_Rate': [volumetrics_count / total_count * 100 if total_count > 0 else 0],
        'Registration_Count': [registration_count],
        'Registration_Rate': [registration_count / total_count * 100 if total_count > 0 else 0],
        'DL_Count': [dl_count],
        'DL_Rate': [dl_count / total_count * 100 if total_count > 0 else 0],
        'Multiple_Failures_Count': [multiple_failures_count],
        'Multiple_Failures_Rate': [multiple_failures_count / total_count * 100 if total_count > 0 else 0]
    })
    
    cta_results.append(df)

# Combine all CTA results
final_cta_results = pd.concat(cta_results, ignore_index=True)

# Define column order
column_order = [
    "Method",
    "Total_Count",
    "Total_Failure_Count", "Total_Failure_Rate",
    "Volumetrics_Count", "Volumetrics_Rate",
    "Registration_Count", "Registration_Rate",
    "DL_Count", "DL_Rate",
    "Multiple_Failures_Count", "Multiple_Failures_Rate"
]

final_cta_results = final_cta_results[column_order]
final_cta_results.to_csv('/Users/rushil/brain_extraction/results/qualitative/scan_lvl/Rushil_QC_cta_results.csv', index=False)
print("CTA results saved to CSV file")
final_cta_results

Processing CTA results for method: CTBET
Found 3 CTA cases for CTBET
Available failure columns for CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing CTA results for method: Robust-CTBET
Found 0 CTA cases for Robust-CTBET
Available failure columns for Robust-CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing CTA results for method: SynthStrip
Found 4 CTA cases for SynthStrip
Available failure columns for SynthStrip: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing CTA results for method: HD-CTBET
Found 1 CTA cases for HD-CTBET
Available failure columns for HD-CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing CTA results for method: CT_BET
Found 6 CTA cases for CT_BET
Available failure columns for CT_BET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing CTA results for method: Brainchop
Filtered out 7 files from brainchop
Found 2 CTA cases for Brainchop
Available failure columns for Brainchop: ['1 - Volumetrics'

,Method,Total_Count,Total_Failure_Count,Total_Failure_Rate,Volumetrics_Count,Volumetrics_Rate,Registration_Count,Registration_Rate,DL_Count,DL_Rate,Multiple_Failures_Count,Multiple_Failures_Rate
0,CTBET,10,3,30.000000,3,30.000000,2,20.000000,2,20.000000,2,20.000000
1,Robust-CTBET,10,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
2,SynthStrip,10,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
3,HD-CTBET,10,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
4,CT_BET,7,5,71.428571,5,71.428571,2,28.571429,2,28.571429,3,42.857143
5,Brainchop,10,1,10.000000,1,10.000000,0,0.000000,0,0.000000,0,0.000000
6,CTbet_Docker,10,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000


**ARTIFACT RESULTS**:

In [13]:
methods = ['CTBET', 'Robust-CTBET', 'SynthStrip', 'HD-CTBET', 'CT_BET', 'Brainchop', 'CTbet_Docker']
exclude_prefixes = ("6046", "6084", "6096", "6246", "6315", "6342", "6499")

brainchop_exclude = ['6109-317_20150302_0647_ct.png', '6142-308_20150610_0707_ct.png', '6193-324_20150924_1431_ct.png', '6257-335_20160118_1150_ct.png',
                     '6418-193_20161228_1248_ct.png', '6470-296_20170602_0607_ct.png', '6480-154_20170622_0937_ct.png']

dockerctbet_exclude = ['6109-317_20150302_0647_ct.png', '6142-308_20150610_0707_ct.png', '6193-324_20150924_1431_ct.png', '6257-335_20160118_1150_ct.png',
                       '6418-193_20161228_1248_ct.png', '6470-296_20170602_0607_ct.png', '6480-154_20170622_0937_ct.png']

artifact_results = []

for method in methods:
    print(f"Processing artifact results for method: {method}")
    # Read the CSV file with the new columns
    csv_file = pd.read_csv(f'/Users/rushil/brain_extraction/data/qc/{method}/{method}_final_v2.csv')
    
    if method == 'brainchop':
        csv_file = csv_file[~csv_file['Filename'].isin(brainchop_exclude)]
        print(f"Filtered out {len(brainchop_exclude)} files from brainchop")
        
    if method == 'CTbet_Docker':
        csv_file = csv_file[~csv_file['Filename'].isin(dockerctbet_exclude)]
        print(f"Filtered out {len(dockerctbet_exclude)} files from dockerctbet")    

    # Apply exclusions based on filename prefixes
    mask = ~csv_file['Filename'].str.startswith(exclude_prefixes)
    csv_file = csv_file[mask]
    
    # Filter for artifact cases only
    artifact_cases = csv_file[csv_file['artifact'] == 1]
    print(f"Found {len(artifact_cases)} artifact cases for {method}")
    
    # Define the failure columns to check
    failure_columns = ['1 - Volumetrics', '2 - Registration', '3 - DL']
    available_columns = [col for col in failure_columns if col in artifact_cases.columns]
    print(f"Available failure columns for {method}: {available_columns}")
    
    # Set total count based on method
    if method != 'CT_BET':
        total_count = 53
    else:
        total_count = 28
    
    # Count failures for each individual column
    volumetrics_count = len(artifact_cases[artifact_cases['1 - Volumetrics'] == 'yes']) if '1 - Volumetrics' in available_columns else 0
    registration_count = len(artifact_cases[artifact_cases['2 - Registration'] == 'yes']) if '2 - Registration' in available_columns else 0
    dl_count = len(artifact_cases[artifact_cases['3 - DL'] == 'yes']) if '3 - DL' in available_columns else 0
    
    # Count total failures (any failure in any column)
    total_failures_count = len(artifact_cases[artifact_cases[available_columns].eq('yes').any(axis=1)]) if available_columns else 0
    
    # Count multiple failures (failures in 2 or more columns)
    yes_counts_per_row = artifact_cases[available_columns].eq('yes').sum(axis=1) if available_columns else pd.Series([0] * len(artifact_cases))
    multiple_failures_count = len(artifact_cases[yes_counts_per_row >= 2])
    
    # Create results dataframe
    df = pd.DataFrame({
        'Method': [method],
        'Total_Count': [total_count],
        'Total_Failure_Count': [total_failures_count],
        'Total_Failure_Rate': [total_failures_count / total_count * 100 if total_count > 0 else 0],
        'Volumetrics_Count': [volumetrics_count],
        'Volumetrics_Rate': [volumetrics_count / total_count * 100 if total_count > 0 else 0],
        'Registration_Count': [registration_count],
        'Registration_Rate': [registration_count / total_count * 100 if total_count > 0 else 0],
        'DL_Count': [dl_count],
        'DL_Rate': [dl_count / total_count * 100 if total_count > 0 else 0],
        'Multiple_Failures_Count': [multiple_failures_count],
        'Multiple_Failures_Rate': [multiple_failures_count / total_count * 100 if total_count > 0 else 0]
    })
    
    artifact_results.append(df)

# Combine all artifact results
final_artifact_results = pd.concat(artifact_results, ignore_index=True)

# Define column order
column_order = [
    "Method",
    "Total_Count",
    "Total_Failure_Count", "Total_Failure_Rate",
    "Volumetrics_Count", "Volumetrics_Rate",
    "Registration_Count", "Registration_Rate",
    "DL_Count", "DL_Rate",
    "Multiple_Failures_Count", "Multiple_Failures_Rate"
]

final_artifact_results = final_artifact_results[column_order]
final_artifact_results.to_csv('/Users/rushil/brain_extraction/results/qualitative/scan_lvl/Rushil_QC_artifact_results.csv', index=False)
print("Artifact results saved to CSV file")
final_artifact_results

Processing artifact results for method: CTBET
Found 12 artifact cases for CTBET
Available failure columns for CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing artifact results for method: Robust-CTBET
Found 2 artifact cases for Robust-CTBET
Available failure columns for Robust-CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing artifact results for method: SynthStrip
Found 47 artifact cases for SynthStrip
Available failure columns for SynthStrip: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing artifact results for method: HD-CTBET
Found 14 artifact cases for HD-CTBET
Available failure columns for HD-CTBET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing artifact results for method: CT_BET
Found 20 artifact cases for CT_BET
Available failure columns for CT_BET: ['1 - Volumetrics', '2 - Registration', '3 - DL']
Processing artifact results for method: Brainchop
Found 24 artifact cases for Brainchop
Available failure columns for 

,Method,Total_Count,Total_Failure_Count,Total_Failure_Rate,Volumetrics_Count,Volumetrics_Rate,Registration_Count,Registration_Rate,DL_Count,DL_Rate,Multiple_Failures_Count,Multiple_Failures_Rate
0,CTBET,53,9,16.981132,9,16.981132,6,11.320755,7,13.207547,7,13.207547
1,Robust-CTBET,53,2,3.773585,2,3.773585,2,3.773585,2,3.773585,2,3.773585
2,SynthStrip,53,30,56.603774,30,56.603774,18,33.962264,22,41.509434,23,43.396226
3,HD-CTBET,53,3,5.660377,3,5.660377,0,0.000000,0,0.000000,0,0.000000
4,CT_BET,28,17,60.714286,17,60.714286,12,42.857143,11,39.285714,14,50.000000
5,Brainchop,53,22,41.509434,22,41.509434,19,35.849057,7,13.207547,19,35.849057
6,CTbet_Docker,53,3,5.660377,3,5.660377,2,3.773585,1,1.886792,3,5.660377
